In [ ]:
# Importing the necessary dependencies
import torch
# checking the device capabilities
major_version, minor_version = torch.cuda.get_device_capability()
# importing unsloth from github and installing
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-o0zu7772/unsloth_949399dd1879480185227c4df6bc4ae6
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-o0zu7772/unsloth_949399dd1879480185227c4df6bc4ae6
  Resolved https://github.com/unslothai/unsloth.git to commit 976d11a10d54383aeb7a692c69e01151a20bfd72
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install triton



In [ ]:
!pip uninstall xformers -y
!pip install xformers==0.0.27

Found existing installation: xformers 0.0.27.post2
Uninstalling xformers-0.0.27.post2:
  Successfully uninstalled xformers-0.0.27.post2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.7 MB/

In [ ]:
# Importing the model
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# Loading the llama3 model as the inference model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [ ]:
# The datasets library that will import our training data
!pip install datasets

In [ ]:
# Importing the Supervised Fintuning Trainer
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
max_seq_length = 2048

# url = "https://huggingface.co/datasets/laion/OIG/resolve/main/unified_chip2.jsonl"
# # Loading the dataset from the hugging face repository
# dataset = load_dataset("json", data_files = {"train" : url}, split = "train")

dataset = load_dataset("DEEPAK70681/CryptoSendIntentv2", split = "train")


Generating train split:   0%|          | 0/1075 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/148 [00:00<?, ? examples/s]

In [ ]:
print(dataset[:5])

{'text': ['<|begin_of_text|><|start_header_id|>user<|end_header_id|>Send 0.5 ETH to 0x1a2B3c4D5E6F7G8H9I0J1K2L3M4N5O6P7Q8R9STU<|eot_id|><|start_header_id|>assistant<|end_header_id|>{\n  "intent": "send",\n  "entities": {\n    "amount": "0.5",\n    "currency": "ETH",\n    "destination_address": "0x1a2B3c4D5E6F7G8H9I0J1K2L3M4N5O6P7Q8R9STU"\n  }\n}<|eot_id|><|end_of_text|>', '<|begin_of_text|><|start_header_id|>user<|end_header_id|>Transfer 200 OP to 0x9Q8R7P6O5N4M3L2K1J0I9H8G7F6E5D4C3B2A1ZXY<|eot_id|><|start_header_id|>assistant<|end_header_id|>{\n  "intent": "send",\n  "entities": {\n    "amount": "200",\n    "currency": "OP",\n    "destination_address": "0x9Q8R7P6O5N4M3L2K1J0I9H8G7F6E5D4C3B2A1ZXY"\n  }\n}<|eot_id|><|end_of_text|>', '<|begin_of_text|><|start_header_id|>user<|end_header_id|>Send 1.5 BNB to 0xABCDEFabcdefABCDEFabcdefABCDEFabcdef1234<|eot_id|><|start_header_id|>assistant<|end_header_id|>{\n  "intent": "send",\n  "entities": {\n    "amount": "1.5",\n    "currency": "BNB",\n

In [ ]:
def generate_text(text):
    inputs = tokenizer(text, return_tensors="pt").to("cuda:0")
    outputs = model.generate(**inputs, max_new_tokens=100)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# Test Pre-train:
print("Before training\n")
FastLanguageModel.for_inference(model)
generate_text("Send 0.25 BTC to 0xdBA0578eAE8406D32ad5Ed362A24e63893535a5e")


Before training

Send 0.25 BTC to 0xdBA0578eAE8406D32ad5Ed362A24e63893535a5e. Please confirm receipt before sending more.
Send 0.25 BTC to 0xdBA0578eAE8406D32ad5Ed362A24e63893535a5e. Please confirm receipt before sending more.
Send 0.25 BTC to 0xdBA0578eAE8406D32ad5Ed362A24e63893535a5e. Please confirm receipt before sending more.
Send 0.25 BTC to 0xdBA057


In [ ]:
# 4. Do model patching and add fast LoRA weights and training
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length,
    use_rslora = False,  # Rank stabilized LoRA
    loftq_config = None, # LoftQ
)
# 5. using the supervised fine tuning trainer to fine tune the model
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size=8,  # Increase batch size if memory permits
        gradient_accumulation_steps=2,  # Adjust based on batch size
        warmup_steps=100,               # Increased warmup steps
        max_steps=500,                  # Increased steps
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,               # Adjusted logging frequency
        output_dir="outputs",
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,

    ),
)
trainer.train()

Unsloth: Already have LoRA adapters! We shall skip this step.


Map:   0%|          | 0/1075 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,075 | Num Epochs = 8
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 2
\        /    Total batch size = 16 | Total steps = 500
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,2.219200
20,2.894800
30,2.068500
40,2.590700
50,1.450800
60,2.046000
70,1.775900
80,2.113300
90,1.701000
100,2.229200


TrainOutput(global_step=500, training_loss=2.2038651657104493, metrics={'train_runtime': 1677.0149, 'train_samples_per_second': 4.77, 'train_steps_per_second': 0.298, 'total_flos': 3.567031552197427e+16, 'train_loss': 2.2038651657104493, 'epoch': 7.407407407407407})

In [ ]:
# Test after training
print("\n ######## \nAfter training\n")
FastLanguageModel.for_inference(model)
generate_text("<|begin_of_text|><|start_header_id|>user<|end_header_id|>Send 59 CPX to 0x1a2B3c4D5E6F7G8H9I0J1K2L3M4N5O6P7Q8R9STU<|eot_id|>")



 ######## 
After training

userSend 59 CPX to 0x1a2B3c4D5E6F7G8H9I0J1K2L3M4N5O6P7Q8R9STUassistant{
  "intent": "send",
  "entities": {
    "amount": "59",
    "currency": "CPX",
    "destination_address": "0x1a2B3c4D5E6F7G8H9I0J1K2L3M4N5O6P7Q8R9STU"
  }
}


In [ ]:
model.save_pretrained("sendIntentModelTHEOGv1")

HuggingFace>Settings>AccessTokens>copy the dataset_model(WRITE)

In [ ]:
!pip install huggingface_hub

In [ ]:

!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
model.push_to_hub("DEEPAK70681/sendIntentModelTHEOGv1")

Saved model to https://huggingface.co/DEEPAK70681/sendIntentModelTHEOGv1


In [ ]:
from google.colab import files

# Download the file
files.download('/content/sendIntentModelTHEOGv1/checkpoint-500/scheduler.pt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>